# Google Personal (Gmail) Authentication Test

This notebook tests the Gmail app password authentication flow for personal Gmail accounts.

In [ ]:
# Add parent directory to path
import sys
sys.path.insert(0, '..')

# Import syft_client
import syft_client
print(f"Syft Client Version: {syft_client.__version__}")

## Test 1: Personal Gmail Login with App Password

This test demonstrates logging into a personal Gmail account using an app password.

In [ ]:
# Test with a personal Gmail account
# NOTE: You'll need to generate an app password for your account

# Option 1: Let it auto-detect as gmail
email = "your.email@gmail.com"  # Replace with your Gmail

try:
    # This will detect it as google_personal and prompt for app password
    result = syft_client.login(email=email, verbose=True)
    print("\nLogin successful!")
    print(f"Auth method: {result.get('auth_method')}")
    print(f"Servers configured: {list(result.get('servers', {}).keys())}")
except Exception as e:
    print(f"Login failed: {e}")

## Test 2: Direct Platform Client Usage

Test using the GooglePersonalClient directly for more control.

In [ ]:
from syft_client.platforms.google_personal import GooglePersonalClient
from syft_client.platforms import Platform

# Create client directly
email = "your.email@gmail.com"  # Replace with your Gmail
client = GooglePersonalClient(email)

# Check login complexity
print(f"Platform: {client.platform}")
print(f"Login complexity: {client.login_complexity} steps")
print(f"Available transport layers: {client.get_transport_layers()}")

# Attempt authentication
try:
    auth_result = client.authenticate()
    print("\n✅ Authentication successful!")
except Exception as e:
    print(f"\n❌ Authentication failed: {e}")

## Test 3: Using Gmail Transport

Once authenticated, test sending and receiving emails.

In [ ]:
# This assumes you've already authenticated above
from syft_client.platforms.google_personal.gmail import GmailTransport

# Create transport with credentials from auth
if 'auth_result' in locals() and auth_result:
    gmail = GmailTransport(auth_result['credentials'])
    
    # Test sending an email to yourself
    test_email = auth_result['email']
    test_data = {
        'message': 'Test from syft_client',
        'timestamp': str(datetime.now()),
        'test': True
    }
    
    print(f"Sending test email to {test_email}...")
    success = gmail.send(
        recipient=test_email,
        data=test_data,
        subject="Syft Client Test Email"
    )
    
    if success:
        print("✅ Email sent successfully!")
    else:
        print("❌ Failed to send email")
    
    # Test receiving emails
    print("\nReceiving recent emails...")
    messages = gmail.receive(limit=5)
    
    print(f"\nFound {len(messages)} recent messages:")
    for msg in messages:
        print(f"- From: {msg['from'][:50]}...")
        print(f"  Subject: {msg['subject']}")
        print(f"  Date: {msg['date']}")
        print(f"  Attachments: {len(msg['attachments'])}")
        print()
else:
    print("Please authenticate first (run Test 2)")

## Test 4: Error Handling

Test various error conditions.

In [ ]:
# Test 1: Invalid app password length
print("Test 1: Invalid app password")
print("-" * 40)

# This will fail because app passwords must be 16 characters
# (The actual test would require interactive input, so this is just a demo)

# Test 2: Wrong platform detection
print("\nTest 2: Corporate email with personal auth")
print("-" * 40)

try:
    # This should be detected as google_org, not google_personal
    result = syft_client.login(email="user@stanford.edu", verbose=True)
except Exception as e:
    print(f"Expected error: {type(e).__name__}")

## Summary

The Google Personal authentication flow:

1. **Requirements**: 
   - Personal Gmail account (gmail.com or googlemail.com)
   - 2-Factor Authentication enabled
   - App password generated

2. **Authentication Steps**:
   - User provides email
   - System detects it as google_personal
   - User chooses app password method
   - User enters 16-character app password
   - System tests SMTP and IMAP connections
   - Credentials saved for future use

3. **Benefits**:
   - Simple one-time setup
   - No browser required
   - Works in all environments
   - Passwords don't expire (unless main password changes)

4. **Limitations**:
   - Requires 2FA to be enabled
   - Less secure than OAuth2
   - Passwords revoked if main password changes